In [ ]:
# This project is on Gas Classification for 11 gases, using 100 GB of data obtained from sensor modules.
# Each sensor module included 8 sensors and was placed in 6 horizontal and 9 vertical locations for measurements under 11 different target gases.
# The goal is to predict the target gas out of 11 possibilities, using the raw data file received from the sensor module.

# Import necessary libraries
import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, auc, precision_recall_curve
import glob
import os
from pathlib import Path


# Download the data from https://archive.ics.uci.edu/dataset/251/gas+sensor+arrays+in+open+sampling+settings

# Create a list of file paths
root_path = 'path/to/your/data/directory'
paths = [os.path.join(root, subfolder) for root, _, subfolders in os.walk(root_path) for subfolder in subfolders]


# Create a list of target gases
gas_list = [folder for folder in os.listdir(root_path) if os.path.isdir(os.path.join(root_path, folder))]


# Create a folder for results
results_dir = os.path.abspath(os.path.join(root_path, os.pardir, 'results'))
os.makedirs(results_dir, exist_ok=True)


# Define the processed_data array
processed_data = np.zeros((0, 12), dtype='int8')


# Make a loop for all raw data files
for file in paths:
    raw_data = pd.read_csv(file, delimiter='\t', header=None, usecols=[filecol for filecol in range(1, 92)],
                           engine='python', dtype='int16')


    # Define a new_data array for the new raw data file
    new_data = np.zeros((1, 12), dtype='int8')

    
    # Find the common parameters for each data file
    fan = np.mean(raw_data[1])
    temperature = np.mean(raw_data[9])
    RH = np.mean(raw_data[10])
    gas_number = 1 + gas_list.index(file.split(os.path.sep)[-3])

    new_data[0, 0] = temperature
    new_data[0, 1] = RH
    new_data[0, 2] = fan
    new_data[0, 11] = gas_number

    
    # Calculate the responses of 8 sensors
    for vertical_location in range(9):
        col = 12 + 9 * vertical_location
        for sensor_number in range(8):
            r1 = np.max(raw_data[col + sensor_number])
            r2 = np.min(raw_data[col + sensor_number])
            new_data[0, sensor_number + 3] = 100 * (r2 - r1) / r1
        processed_data = np.concatenate([processed_data, new_data])


# Convert the processed_data array to a dataframe
processed_data_df = pd.DataFrame(processed_data).astype('int8')


# Save the processed data dataframe as a CSV, overwriting the previous CSV file
savepath = os.path.join(results_dir, 'processed_data.csv')
processed_data_df.to_csv(savepath, mode='w', index=False, header=None)


# Define features and labels, and split the data
x = processed_data_df.iloc[:, 0:-1]  # Features
y = processed_data_df.iloc[:, -1]    # Labels
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.3, shuffle=True)


# Display the sizes of the train and test datasets
print("Training feature matrix shape:", x_train.shape)
print("Testing feature matrix shape:", x_test.shape)
print("Training target matrix shape:", y_train.shape)
print("Testing target matrix shape:", y_test.shape)



# Model building
clf = RandomForestClassifier(n_estimators=50, max_depth=200, criterion='entropy', random_state=1)
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)



# Model evaluation
true_labels = y_test
predicted_labels = y_pred


# Classification Report
report = classification_report(true_labels, predicted_labels)
print(report)


# Visualize Confusion Matrix
cm = confusion_matrix(true_labels, predicted_labels)
sns.heatmap(cm, annot=True, fmt="d")
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.show()


# ROC and AUC

# Create a multiclass classifier 
clf_ovr = OneVsRestClassifier(RandomForestClassifier(n_estimators=50, max_depth=200, 
                                                        criterion='entropy', random_state=1))

# Fit the classifier
clf_ovr.fit(x_train, y_train)


# Calculate the predicted probabilities
predicted_probs = clf_ovr.predict_proba(x_test)


# Convert numerical labels to a binary indicator matrix
true_labels_bin = label_binarize(y_test, classes=np.unique(y))


# Initialize a dictionary to store the ROC AUC scores for each class
roc_auc = {}


# Plot ROC curve for each class
plt.figure(figsize=(8, 6))
n_classes = true_labels_bin.shape[1]
for i in range(n_classes):
    fpr, tpr, _ = roc_curve(true_labels_bin[:, i], predicted_probs[:, i])
    roc_auc[i] = metrics.auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f'Class {i+1} (AUC = {roc_auc[i]:.2f})')

# Add the diagonal line (random classifier)
plt.plot([0, 1], [0, 1], 'k--')

# Set plot properties
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Multiclass Receiver Operating Characteristic (OvR)')
plt.legend(loc="lower right")
plt.show()


# Plot Precision-Recall Curve for each class
plt.figure(figsize=(8, 6))
for i in range(n_classes):
    precision, recall, _ = precision_recall_curve(true_labels_bin[:, i], predicted_probs[:, i])
    plt.plot(recall, precision, label=f'Class {i+1}')
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend(loc='best')
plt.show()